Task-4.9 steps 1 - 3

In [1]:
#Importing Libraries 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

#Path setup and Importing Dataframes

In [2]:
path=r"C:\Users\ranji\Downloads\Submitted tasks\Python_5\Instacart\Instacart-Basket-Analysis"

In [3]:
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

#Importing orders_products_merged_grouping.pkl dataframe

In [4]:
ords_prods_group = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_grouping.pkl'))

Step - 4 Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that don’t add anything to your analysis.

In [5]:
customers.columns
#Inspect the column names in the customers.csv dataframe to identify any unnecessary or illogical names

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [6]:
customers.shape
#Checking the size of the dataframe customers.csv

(206209, 10)

In [7]:
customers.info()
#checking the customers information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


#Observation: The "First Name" column has 194,950 non-null entries, while the "Surname" column has 206,209 entries. This indicates that there are 206,209 - 194,950 = 11,259 empty cells. To confirm this, we can use the value_counts() function to count the occurrences of each value in the "Surname" column, including empty cells.

In [9]:
customers['First Name'].value_counts(dropna = False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

#To ensure the accuracy of the data in the "First Name" column, we will perform a double check for any missing values. As this column has fewer non-null entries compared to others, it is important to verify its integrity.

In [10]:
customers.describe()
#Calculating the descriptive statistics of the "customers.csv" dataset.

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


Observations: Approximately 5.5% of the first names are missing, which accounts for 11,259 entries. Since we lack information about these names, leaving them blank should not significantly impact the overall statistics of the dataset.

In [11]:
customers.head(10)
#Checking the first ten rows of customers.csv

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


Observations: One suggestion would be to rename the column 'STATE' to 'State' to follow a consistent capitalization style. However, apart from that, there doesn't seem to be a need to remove any columns at this time.

In [12]:
customers.rename(columns = {'STATE' : 'State'}, inplace = True)

In [13]:
customers.head(5)

,user_id,First Name,Surnam,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Step - 5 Check for and address missing values and duplicates, and convert any mixed-type data.

In [14]:
#This code helps to identify columns where some values may have been stored with different data types. 

for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis=1)
    if len(customers[weird]) > 0:
        print(col)

First Name


Observations: The 'First Name' column contains null values, which is causing the mixed data type. The rest of the columns appear to be fine. To address this, we can convert the data type of the 'First Name' column to string, ensuring consistency throughout the column.

Change the data type of the "First Name" column to string.

In [15]:
customers['First Name'] = customers['First Name'].astype('str')
print(customers.dtypes)

user_id          int64
First Name      object
Surnam          object
Gender          object
State           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object


#Checking for mixed types in the DataFrame:

In [16]:
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis=1)
    if len(customers[weird]) > 0:
        print("Inconsistent data types found in column:", col)

#No inconsistencies are found

#Checking for missing values

In [17]:
customers.isnull().sum()

user_id         0
First Name      0
Surnam          0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

#There are no missing values in the dataset.

#checking for duplicate values 

In [18]:
duplicates = customers.duplicated()
if duplicates.any():
    print("Duplicates found!")
else:
    print("No duplicates found.")

No duplicates found.


step -  6 Combine your customer data with the rest of your prepared Instacart data.

In [19]:
ords_prods_group.info()

##Checking ords_prods_group

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434212 entries, 0 to 32434211
Data columns (total 24 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   eval_set                object  
 3   order_number            int64   
 4   orders_day_of_week      int64   
 5   order_hour_of_day       int64   
 6   days_since_prior_order  float64 
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  product_name            object  
 11  aisle_id                int64   
 12  department_id           int64   
 13  prices                  float64 
 14  _merge                  category
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  average_price           float64 
 21  spendi

In [20]:
#Checking the ords_prods_group size

ords_prods_group.shape

(32434212, 24)

In [21]:
#Re-checking the customers.csv info

customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    206209 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   State         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [22]:
#Combining the two df using the user_id column

combined = ords_prods_group.merge(customers, on = 'user_id')

In [23]:
#Checking first five rows of combined df

combined.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,order_frequency_flag,First Name,Surnam,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [24]:
#Checking basic info of combined df to ensure information is correct

combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434212 entries, 0 to 32434211
Data columns (total 33 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   eval_set                object  
 3   order_number            int64   
 4   orders_day_of_week      int64   
 5   order_hour_of_day       int64   
 6   days_since_prior_order  float64 
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  product_name            object  
 11  aisle_id                int64   
 12  department_id           int64   
 13  prices                  float64 
 14  _merge                  category
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  average_price           float64 
 21  spendi

step - 7 Ensure your notebook contains logical titles, section headings, and descriptive code comments.

Done

In [ ]:
step - 8 Export this new dataframe as a pickle file so you can continue to use it in the second part of this task.

In [25]:
#Exporting combined customer data set with ords_prods_group to pickle file

combined.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customer_ords_prods_merged.pkl'))

In [ ]:
step - 9 Save your notebook so that you can send it to your tutor for review after completing part 2.